# Movie Recommendation

In [6]:
import json
import sys
import numpy
import ipywidgets as widgets

sys.path.append("../")

from common.compute_scores import pearson
from common.collaborative_filtering import find_similar_users

## Input Data

In [7]:
with open("ratings.json", "r") as file:
    dataset = json.load(file)

users = tuple(sorted(dataset.keys()))

## Making Recommendations

In [12]:
def get_recommendations(data, user):
    # Will not recommend movies the user has already rated
    overall_scores = {}
    similarity_scores = {}
    
    for u in (x for x in data if x != user):
        # For each other user...
        similarity = pearson(data, user, u)
        
        if similarity <= 0:
            # If these users aren't similar in some way...
            continue
        
        candidates = [x for x in data[u] if x not in data[user] or data[user][x] == 0]
        # List of movies rated by the current user, but not the input user
        # I.e. we're not sure if the input user will like this one yet
        
        for movie in candidates:
            overall_scores.update({ movie: data[u][movie] * similarity })
            similarity_scores.update({ movie: similarity })
    
    if not overall_scores:
        # If we couldn't find any movies this user might like...
        return []

    movie_scores = numpy.array([[ score / similarity_scores[movie], movie] for movie, score in overall_scores.items()])
    movie_scores = movie_scores[numpy.argsort(movie_scores[:, 0])[::-1]]
    recommendations = [movie for _, movie in movie_scores]
    
    return recommendations


## Displaying Results

In [11]:
def recommend(user):
    recommendations = get_recommendations(dataset, user)
    
    print("Movie Recommendations for {0}:".format(user))
    
    if recommendations:
        for movie in recommendations:
            print("\t- {0}".format(movie))
    else:
        print("\t- None Found")

users = tuple(sorted(dataset.keys()))

user = widgets.Dropdown(options=users, description="User")

widgets.interactive(
    recommend,
    user=user
)

interactive(children=(Dropdown(description='User', options=('Adam Cohen', 'Bill Duffy', 'Brenda Peterson', 'Ch…